In [ ]:
from collections import Counter
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import sys
import os

# Define root directory
# root = os.path.dirname(os.path.abspath(__file__))
root = "."
sys.path.append(os.path.join(root, "..", "src"))
from default import CONFIGPATH, MIN_ASSAY_SIZE

# List of pathogens to process
pathogens = ["Acinetobacter baumannii", "Candida albicans", "Campylobacter", "Escherichia coli", "Enterococcus faecium", "Enterobacter",
             "Helicobacter pylori", "Klebsiella pneumoniae", "Mycobacterium tuberculosis", "Neisseria gonorrhoeae", "Pseudomonas aeruginosa",
             "Plasmodium falciparum", "Staphylococcus aureus", "Schistosoma mansoni", "Streptococcus pneumoniae"][8:9]

def get_pathogen_code(pathogen):
    return str(pathogen.split()[0][0] + pathogen.split()[1]).lower() if len(pathogen.split()) > 1 else pathogen.lower()

# Helper function - is there only a single value?
def only_one(values, name):
    if len(values) != 1:
        raise ValueError(f"Expected exactly one {name}, found {values}")
    return values[0]

# Create output directory
OUTPUT = os.path.join(root, "..", "output")

# For each pathogen
for pathogen in pathogens:

    print("\n\n\n")

    # Loading pathogen data
    pathogen_code = get_pathogen_code(pathogen)
    print(f"Loading ChEMBL preprocessed data for {pathogen_code}...")
    ChEMBL_pathogen = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, f"{pathogen_code}_ChEMBL_raw_data.csv.gz"), low_memory=False)
    print(f"Number of activities for {pathogen_code}: {len(ChEMBL_pathogen)}")
    print(f"Number of compounds for {pathogen_code}: {len(set(ChEMBL_pathogen['compound_chembl_id']))}")
    ASSAYS_RAW = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, 'assays_raw.csv'))
    print(f"Original number of assays: {len(ASSAYS_RAW)}")

    # Converting activity types to their corresponding synonyms
    synonyms = pd.read_csv(os.path.join(root, "..", "config", "manual_curation", "synonyms.csv"))
    for activity, syns in zip(synonyms['activity'], synonyms['synonyms']):
        for syn in syns.split(";"):
            ChEMBL_pathogen.loc[ChEMBL_pathogen['activity_type'] == syn, 'activity_type'] = activity

    # Discard activities with no value nor act/inact flag in activity_comment not standard_text
    ChEMBL_pathogen = ChEMBL_pathogen[(ChEMBL_pathogen['value'].isna() == False) | 
                                    (ChEMBL_pathogen['activity_comment'] != 0) | 
                                    (ChEMBL_pathogen['standard_text'] != 0)].reset_index(drop=True)
    
    print(f"Removing activities with no value nor act/inact flag in activity_comment nor standard_test...")
    print(f"Number of activities for {pathogen_code}: {len(ChEMBL_pathogen)}")
    print(f"Number of compounds for {pathogen_code}: {len(set(ChEMBL_pathogen['compound_chembl_id']))}")

    # Get directions
    DIRECTIONS = pd.read_csv(os.path.join(root, "..", "config", 'manual_curation', 'activity_std_units_curated_manual_curation.csv'))
    DIRECTIONS = {(i,j): k for i,j,k in zip(DIRECTIONS['activity_type'], DIRECTIONS['unit'], DIRECTIONS['manual_curation']) if np.isnan(k) == False}
    ChEMBL_pathogen['direction'] = [DIRECTIONS[(i,j)] if (i,j) in DIRECTIONS else np.nan 
                                    for i,j in zip(ChEMBL_pathogen['activity_type'], ChEMBL_pathogen['unit'])]
    print(f"Directions assigned. Summary: {Counter(ChEMBL_pathogen['direction'].fillna('NaN'))}")

    # Identify canonical unit per activity type
    print("Identifying canonical unit per activity type...")
    # Get pair counts
    s = ChEMBL_pathogen[["activity_type", "unit"]]
    out = (
    s.value_counts(subset=["activity_type", "unit"], dropna=False)
        .reset_index(name="count")
        .sort_values("count", ascending=False, ignore_index=True))

    # Identify the most occurring pairs
    idx = out.groupby("activity_type")['count'].idxmax()
    out["canonical_unit"] = False
    out.loc[idx, "canonical_unit"] = True
    print(f"Number of unique activity type - unit pairs: {len(out)}")

    # Get canonical unit per activity type
    canonical = (
        out[out["canonical_unit"] == 1]
        .set_index("activity_type")[["unit"]])
    canonical_map = canonical["unit"].to_dict()
    ChEMBL_pathogen["canonical_unit"] = ChEMBL_pathogen["activity_type"].map(canonical_map)

    # Save pair summary
    out.to_csv(os.path.join(root, "..", "output", pathogen_code, "activity_type_unit_pairs.csv"), index=False)

    # Save cleaned data
    ChEMBL_pathogen.to_csv(os.path.join(root, "..", "output", pathogen_code, f"{pathogen_code}_ChEMBL_cleaned_data.csv.gz"), index=False)

    # Get unique assays
    assays = sorted(set(ChEMBL_pathogen['assay_chembl_id']))

    ASSAYS_INFO = []
    print("Collecting individual assay information...")
    print(f"Number of unique assays: {len(assays)}")

    # Load standardized units 
    standardized_units = set(pd.read_csv(os.path.join(CONFIGPATH, "manual_curation", 'ucum_GT.csv'))['final_unit'])

    # For each assay
    for assay in tqdm(assays[:100]):

        # Get subset of strain + assay data
        df_ = ChEMBL_pathogen[ChEMBL_pathogen["assay_chembl_id"] == assay]
        
        # Get values
        assay_type = list(set(df_['assay_type']))
        target_type = list(set(df_['target_type']))
        target_chembl_id = list(set(df_['target_chembl_id']))
        activity_types = list(set(df_['activity_type']))
        target_organism = list(set(df_['target_organism']))
        assay_organism = list(set(df_['assay_organism']))
        doc_chembl_id = list(set(df_['doc_chembl_id']))

        # Check coherence
        assay_type = only_one(assay_type, "assay_type")
        target_type = only_one(target_type, "target_type")
        target_chembl_id = only_one(target_chembl_id, "target_chembl_id")
        target_organism = only_one(target_organism, "target_organism")
        assay_organism = only_one(assay_organism, "assay_organism")
        doc_chembl_id = only_one(doc_chembl_id, "doc_chembl_id")

        # For each activity type
        for act_type in activity_types:

            df__ = df_[df_["activity_type"] == act_type]
            activity_type = list(set(df__['activity_type']))
            activity_type = only_one(activity_type, 'activity_type')
            units = list(set(df__['unit']))

            for u in units:
                if type(u) != str:
                    df___ = df__[df__["unit"].isna()]
                else:
                    df___ = df__[df__["unit"] == u]
                unit = list(set(df___['unit']))
                unit = only_one(unit, "unit")
                activities = len(df___)
                cpds = len(set(df___['compound_chembl_id']))
                nan_values = len(df___[df___['value'].isna()])
                direction = DIRECTIONS[(act_type, unit)] if (act_type, unit) in DIRECTIONS else np.nan
                canonical_unit = canonical_map[act_type]  # this
                is_consensus = unit in standardized_units  # this
                ASSAYS_INFO.append([assay, assay_type, assay_organism, doc_chembl_id, target_type, target_chembl_id, target_organism, 
                                    activity_type, unit, canonical_unit, is_consensus, activities, nan_values, cpds, direction])

    ASSAYS_INFO = pd.DataFrame(ASSAYS_INFO, columns=["assay_id", "assay_type", "assay_organism", "doc_chembl_id", "target_type", "target_chembl_id", "target_organism", 
                                                        "activity_type", "unit", "canonical_unit", "is_consensus", "activities", 'nan_values', "cpds", "direction"])
    ASSAYS_INFO = ASSAYS_INFO.sort_values('cpds', ascending=False).reset_index(drop=True)

    # Filter assays with too few compounds
    ASSAYS_INFO = ASSAYS_INFO[ASSAYS_INFO['cpds'] > MIN_ASSAY_SIZE].reset_index(drop=True)

    # Save assays info
    # ASSAYS_INFO.to_csv(os.path.join(root, "..", "output", pathogen_code, 'assays_cleaned.csv'), index=False)

    break





Loading ChEMBL preprocessed data for mtuberculosis...
Number of activities for mtuberculosis: 714221
Number of compounds for mtuberculosis: 132378
Original number of assays: 13587
Removing activities with no value nor act/inact flag in activity_comment nor standard_test...
Number of activities for mtuberculosis: 705718
Number of compounds for mtuberculosis: 130771
Directions assigned. Summary: Counter({1.0: 605660, -1.0: 91779, 'NaN': 6028, 0.0: 2251})
Identifying canonical unit per activity type...
Number of unique activity type - unit pairs: 198
Number of unique assays: 10902


100%|██████████| 100/100 [00:02<00:00, 36.23it/s]


In [32]:
ChEMBL_pathogen

,activity_id,assay_id,assay_chembl_id,assay_type,assay_confidence_score,assay_organism,doc_chembl_id,tid,target_type,target_organism,...,pchembl,activity_comment,standard_text,value,unit,activity_type,relation,pchembl_calculated,direction,canonical_unit
0,33041,140679,CHEMBL752463,F,1,Mycobacterium tuberculosis,CHEMBL1138695,50309,ORGANISM,Mycobacterium tuberculosis,...,NaN,0,0,0.000000,%,INHIBITION,=,NaN,1.0,%
1,33985,140501,CHEMBL749419,F,1,Mycobacterium tuberculosis,CHEMBL1148365,50309,ORGANISM,Mycobacterium tuberculosis,...,NaN,0,0,1.920000,NaN,PMIC,=,NaN,NaN,NaN
2,33986,140501,CHEMBL749419,F,1,Mycobacterium tuberculosis,CHEMBL1148365,50309,ORGANISM,Mycobacterium tuberculosis,...,NaN,0,0,1.820000,NaN,PMIC,=,NaN,NaN,NaN
3,34021,101206,CHEMBL715914,F,1,Mycobacterium tuberculosis,CHEMBL1124774,50309,ORGANISM,Mycobacterium tuberculosis,...,NaN,0,0,0.717546,umol.L-1,MIC,=,6.144150,-1.0,umol.L-1
4,34115,101206,CHEMBL715914,F,1,Mycobacterium tuberculosis,CHEMBL1124774,50309,ORGANISM,Mycobacterium tuberculosis,...,NaN,0,0,1.071213,umol.L-1,MIC,=,5.970124,-1.0,umol.L-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705713,28559585,2594925,CHEMBL5737624,B,8,NaN,CHEMBL5728864,122090,SINGLE PROTEIN,Mycobacterium tuberculosis (strain ATCC 25618 ...,...,7.85,0,0,0.014000,umol.L-1,IC50,=,7.853872,-1.0,umol.L-1
705714,28559597,2594925,CHEMBL5737624,B,8,NaN,CHEMBL5728864,122090,SINGLE PROTEIN,Mycobacterium tuberculosis (strain ATCC 25618 ...,...,6.11,0,0,0.770000,umol.L-1,IC50,=,6.113509,-1.0,umol.L-1
705715,28559606,2594925,CHEMBL5737624,B,8,NaN,CHEMBL5728864,122090,SINGLE PROTEIN,Mycobacterium tuberculosis (strain ATCC 25618 ...,...,6.17,0,0,0.684000,umol.L-1,IC50,=,6.164944,-1.0,umol.L-1
705716,28559615,2594925,CHEMBL5737624,B,8,NaN,CHEMBL5728864,122090,SINGLE PROTEIN,Mycobacterium tuberculosis (strain ATCC 25618 ...,...,5.90,0,0,1.270000,umol.L-1,IC50,=,5.896196,-1.0,umol.L-1


In [21]:
out

,activity_type,unit,count,canonical_unit
0,PERCENTEFFECT,%,593327,True
1,MIC,umol.L-1,46439,True
2,IC50,umol.L-1,16761,True
3,MIC90,umol.L-1,10579,True
4,INHIBITION,%,7606,True
...,...,...,...,...
193,RANGEOFMIC,umol.L-1,1,True
194,VMAX,RFU/min,1,False
195,VD,L.kg-1,1,True
196,VMAX,U.mg-1,1,False
